In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import matplotlib.cm as cm

In [ ]:
"""TO CHANGE: Neuron count -> synapse count for syn2p analysis"""

In [ ]:
def load(fp):
    stat = np.load(f"{fp}\stat.npy", allow_pickle = True)
    ops = np.load(f"{fp}\ops.npy", allow_pickle = True).item()
    F = np.load(f"{fp}\F.npy", allow_pickle = True)
    Fneu = np.load(f"{fp}\Fneu.npy", allow_pickle = True)
    #iscell / spks also can be repeated in this way
    return stat, ops, F, Fneu
def show_fig(fp):
    stat, ops, F, Fneu = load(fp)
    MaxImg = getMaxImg(ops)
    scatters, nid2idx, nid2idx_rejected = getStats(stat, MaxImg.shape)
    dispPlot(MaxImg, scatters, nid2idx, nid2idx_rejected, F, Fneu)

def getMaxImg(ops):
    maxImg = ops["max_proj"] # Also "max_proj", "meanImg", "meanImgE"
    mimg = maxImg # Use suite-2p source-code naming
    mimg1 = np.percentile(mimg,1)
    mimg99 = np.percentile(mimg,99)
    mimg = (mimg - mimg1) / (mimg99 - mimg1)
    mimg = np.maximum(0,np.minimum(1,mimg))
    mimg *= 255
    mimg = mimg.astype(np.uint8)
    return mimg

from scipy.ndimage import binary_dilation, binary_fill_holes

def boundary(ypix,xpix):
    """ returns pixels of mask that are on the exterior of the mask """
    ypix = np.expand_dims(ypix.flatten(),axis=1)
    xpix = np.expand_dims(xpix.flatten(),axis=1)
    npix = ypix.shape[0]
    if npix>0:
        msk = np.zeros((np.ptp(ypix)+6, np.ptp(xpix)+6), bool) #OG = np.bool, bool is the new un-depreciated version
        msk[ypix-ypix.min()+3, xpix-xpix.min()+3] = True
        msk = binary_dilation(msk)
        msk = binary_fill_holes(msk)
        k = np.ones((3,3),dtype=int) # for 4-connected
        k = np.zeros((3,3),dtype=int); k[1] = 1; k[:,1] = 1 # for 8-connected
        out = binary_dilation(msk==0, k) & msk

        yext, xext = np.nonzero(out)
        yext, xext = yext+ypix.min()-3, xext+xpix.min()-3
    else:
        yext = np.zeros((0,))
        xext = np.zeros((0,))
    return yext, xext

def getStats(stat, frame_shape):
    MIN_SKEW = 2.0 #TODO: define for iscell; also figure exclude noncell
    max_pixel = 200
    max_compact = 1.25
    min_footprint = 0
    max_footprint = 3
    # pixel2neuron = np.full((sum(stat.shape), sum(stat.shape)), fill_value=np.nan, dtype=float)
    scatters = dict(x=[], y=[], color=[], text=[])
    min_skew, max_skew = -20000, 20000 #swapoped min/max 20,000 *-1
    nid2idx = {}
    nid2idx_rejected = {}
    print(f"Num synapses: {stat.shape[0]}")
    # from suite2p.gui.utils import boundary; recreated the function manually
    for n in range(stat.shape[0]):
        skew = stat[n]['skew']
        # min_skew, max_skew = min(skew, min_skew), max(skew, max_skew)
        # npix = stat[n]['npix']
        footprint = stat[n]['footprint']
        compact = stat[n]['compact']
        # min_skew, max_skew = min(skew, min_skew), max(skew, max_skew)
        if skew >= MIN_SKEW and footprint > min_footprint and compact <= max_compact:# and npix <= max_pixel and compact <= max_compact and footprint < max_footprint
            nid2idx[n] = len(scatters["x"]) # Assign new idx
        else:
            nid2idx_rejected[n] = len(scatters["x"])
        ypix = stat[n]['ypix'].flatten() - 1#[~stat[n]['overlap']] - 1
        xpix = stat[n]['xpix'].flatten() - 1#[~stat[n]['overlap']] - 1
        yext, xext = boundary(ypix, xpix)
        scatters['x'] += [xext]
        scatters['y'] += [yext]
        # pixel2neuron[xpix, ypix] = n
        scatters["color"].append(skew)
        scatters["text"].append(f"Cell #{n} - Skew: {skew}")
    print("Min/max skew: ", min_skew, max_skew)
    # Normalize colors between 0 and 1
    color_raw = np.array(scatters["color"])
    scatters["color"] = (color_raw - min_skew) / (max_skew - min_skew)
    return scatters, nid2idx, nid2idx_rejected #, pixel2neuron

def dispPlot(MaxImg, scatters, nid2idx, nid2idx_rejected,
             F, Fneu, axs=None): #pixel2neuron parameter removed here
  if axs is None:
    fig = plt.figure(constrained_layout=True)
    NUM_GRIDS=12
    gs = fig.add_gridspec(NUM_GRIDS, 1)
    ax1 = fig.add_subplot(gs[:NUM_GRIDS-2])
    # ax2 = fig.add_subplot(gs[NUM_GRIDS-2])
    fig.set_size_inches(12,14)
  else:
    ax1 = axs
    ax1.set_xlim(0, MaxImg.shape[0])
    ax1.set_ylim(MaxImg.shape[1], 0)
    # ax2 = None
  ax1.imshow(MaxImg, cmap='gist_gray')
  #fig.update_xaxes(showticklabels=False)
  #fig.update_yaxes(showticklabels=False)
  print("Synapse count:", len(nid2idx))
  norm = matplotlib.colors.Normalize(vmin=0, vmax=1, clip=True)
  mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)

  def plotDict(n2d2idx_dict, override_color = None):
    for neuron_id, idx in n2d2idx_dict.items():
      color = override_color if override_color else mapper.to_rgba(scatters['color'][idx])
    # print(f"{idx}: {scatters['x']} - {scatters['y'][idx]}")
      
      sc = ax1.scatter(scatters["x"][idx], scatters['y'][idx], color = color, 
                     marker='.', s=1)
  plotDict(nid2idx)
  # plotDict(nid2idx_rejected, 'r')
  ax1.set_title(f"Synapses Used: {len(nid2idx)}/{len(nid2idx)+len(nid2idx_rejected)} detected (orange)")

show_fig(r'C:\Users\jcbegs\Desktop\Lab_Business\AG_Dean\calcium_imaging\calcium imaging videos\step1\baseline+NBQX_wtRt_100k_.4ul_hSyn_DIV20_48h_ctrl_60x_NBQX_c1_r003\suite2p\plane0')
print("20fps")

In [1]:
stat, ops, F, Fneu = load(r'C:\Users\jcbegs\Desktop\Lab_Business\frame_test\5fps\suite2p\plane0')

#Here is example code to make an image where each cell (without its overlapping pixels) is a different “number”:

im = np.zeros((ops['Ly'], ops['Lx']))
for n in range(0,stat.shape[0]):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1
plt.imshow(im)
plt.show

NameError: name 'load' is not defined

In [ ]:
"""SANDBOXXX"""


stat, ops, F, Fneu = load(r'C:\Users\jcbegs\Desktop\Lab_Business\AG_Dean\calcium_imaging\suite2p_tiff\frame_rate_test_new\5fps\suite2p\plane0')
scatters, nid2idx, nid2idx_rejected, pixel2neuron = getStats(stat, getMaxImg(ops).shape)
print(scatters, nid2idx, nid2idx_rejected, pixel2neuron)